In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Load features

In [ ]:
df_labeled = pickle.load(open("ip_filtered_features.pkl", "rb"))

In [ ]:
df_labeled

In [ ]:
df_fridge = df_labeled[df_labeled['device'] == 'refrigerator']
df_washer = df_labeled[df_labeled['device'] == 'washer']
df_alexa = df_labeled[df_labeled['device'] == 'alexa']
df_nestcam = df_labeled[df_labeled['device'] == 'nestcam']
df_vacuum = df_labeled[(df_labeled['device'] == 'vacuum') & (df_labeled['environment'] == 'lab')]
df_vacuum_home = df_labeled[(df_labeled['device'] == 'vacuum') & (df_labeled['environment'] == 'home')]

In [ ]:
df_labeled.groupby('activity').size()

In [ ]:
import matplotlib.pyplot as plt
df_labeled['duration'].hist(bins = 100)

# PCA assessment

In [ ]:
feat_sf_array = np.array(df_labeled.feat_sf.tolist())
feat_netml_array = np.array(df_labeled.feat_netml.tolist())

In [ ]:
pca_sf = PCA()
pca_netml = PCA()
Y_sf = pca_sf.fit_transform(feat_sf_array)
Y_netml = pca_netml.fit_transform(feat_netml_array)
act_npy = np.array(df_labeled.activity.tolist())
user_npy = np.array(df_labeled.user.tolist())
device_npy = np.array(df_labeled.device.tolist())

## SlowFast

In [ ]:
plt.scatter(Y_sf[:,0],Y_sf[:,1])

In [ ]:
for l in df_labeled.activity.unique():
    plt.scatter(Y_sf[act_npy==l,0],Y_sf[act_npy==l,1])
plt.legend(df_labeled.activity.unique())

In [ ]:
lset = df_labeled.user.unique()
for l in lset:
    plt.scatter(Y_sf[user_npy==l,0],Y_sf[user_npy==l,1])
plt.legend(lset)

## NetML

In [ ]:
plt.scatter(Y_netml[:,0],Y_netml[:,1])
plt.xlim(-1e5, 3e5)

In [ ]:
for l in df_labeled.activity.unique():
    plt.scatter(Y_netml[act_npy==l,0],Y_netml[act_npy==l,1])
plt.legend(df_labeled.activity.unique())
plt.xlim(-1e5, 3e5)

In [ ]:
lset = df_labeled.user.unique()
for l in lset:
    plt.scatter(Y_netml[user_npy==l,0],Y_netml[user_npy==l,1])
plt.legend(lset)
plt.xlim(-1e5, 3e5)

# Single Modality Modeling

In [ ]:
import numpy as np
import pandas as pd

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def evaluate_model(df_labeled, label, ft_type='feat_sf', verbose=False, by_user=False):
    if ft_type == 'feat_sf':
        classifiers={"OvRLogReg": OneVsRestClassifier(LogisticRegression(random_state=42, max_iter=1000)),}
    elif ft_type == 'feat_netml':
        classifiers={"RF": RandomForestClassifier(random_state=42),}
    if by_user:
        df_labeled = df_labeled.replace('1', '7')
        user_li = df_labeled.user.tolist()
        choice_li = df_labeled.user.unique().tolist()
        for item_ in range(np.shape(choice_li)[0]):
            if choice_li[item_] == '0':
                del choice_li[item_]
        random.seed(42)
        for item_num in range(0, np.shape(user_li)[0]):
            if user_li[item_num] == '0':
                user_li[item_num] = random.choice(choice_li)
        df_labeled['user'] = user_li
        
        if np.shape(df_labeled.user.unique())[0] < 5:
            gkf = GroupKFold(n_splits=np.shape(df_labeled.user.unique())[0])
        else:
            gkf = GroupKFold(n_splits=5)
        index_li = list(gkf.split(df_labeled, groups=df_labeled.user.tolist()))
    else:
        kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
        index_li = list(kf.split(df_labeled))

    if np.shape(df_labeled['device'].unique())[0] > 1:
        device = "combined"
    else:
        device = df_labeled['device'].unique()[0]
    
    df_stats = pd.DataFrame(columns=["classifier", "device", "accuracy", "precision", "recall", "f1_micro", "f1_macro", "f1_weighted", "feature"])
    for key in classifiers:
        clf = classifiers[key]
        Ytrue_li = []
        Ypred_li = []
        for train_index, test_index in index_li:
            df_train = df_labeled.iloc[train_index].sample(frac=0.85, random_state=42)
            df_test = df_labeled.iloc[test_index]
            if ft_type == 'duration':
                clf.fit(np.array(df_train[ft_type].tolist()).reshape(-1, 1), df_train[label].tolist())
            else:
                clf.fit(df_train[ft_type].tolist(), df_train[label].tolist())

            if ft_type == 'duration':
                Ypred = clf.predict(np.array(df_test[ft_type].tolist()).reshape(-1, 1))
            else:
                Ytrue = df_test[label].tolist()
                Ypred = clf.predict(df_test[ft_type].tolist())

                Ytrue_li.append(Ytrue)
                Ypred_li.append(Ypred)
                
                if verbose:
                    result = permutation_importance(
                        clf, df_test[ft_type].tolist(), df_test[label].tolist(), n_repeats=10, random_state=42, n_jobs=2
                    )
                    if ft_type == 'feat_sf':
                        feature_names = ['feature' + str(i) for i in range(400)]
                    elif ft_type == 'feat_netml':
                        feature_names = ["sub_duration_outbound", "pkts_rate_outbound", "bytes_rate_outbound", "np.mean(sizes)_outbound", "np.std(sizes)_outbound", "q1_size_outbound", 
                                           "q2_size_outbound", 'q3_size_outbound', "np.min(sizes)_outbound", 'np.max(sizes)_outbound', "np.mean(iats)_outbound", 'np.std(iats)_outbound', 
                                           "q1_iat_outbound", 'q2_iat_outbound', "q3_iat_outbound", "np.min(iats)_outbound", 'np.max(iats)_outbound', "flow_num_outbound",
                                           "sub_duration_inbound", "pkts_rate_inbound", "bytes_rate_inbound", "np.mean(sizes)_inbound", "np.std(sizes)_inbound", "q1_size_inbound", 
                                           "q2_size_inbound", 'q3_size_inbound', "np.min(sizes)_inbound", 'np.max(sizes)_inbound', "np.mean(iats)_inbound", 'np.std(iats)_inbound', 
                                           "q1_iat_inbound", 'q2_iat_inbound'," q3_iat_inbound", "np.min(iats)_inbound", 'np.max(iats)_inbound', "flow_num_inbound"]

                    importances = pd.Series(result.importances_mean, index=feature_names)
                    print(key, label)
                    print(classification_report(Ytrue, Ypred))
            
            acc = accuracy_score(Ytrue, Ypred)
            pre = precision_score(Ytrue, Ypred, average='weighted')
            rec = recall_score(Ytrue, Ypred, average='weighted')
            f1_micro = f1_score(Ytrue, Ypred, average='micro')
            f1_macro = f1_score(Ytrue, Ypred, average='macro')
            f1_weighted = f1_score(Ytrue, Ypred, average='weighted')
            
            df_stats_this = pd.DataFrame({ "classifier": key, "device": device,
                                          "feature": ft_type, 
                                          "accuracy": acc, "precision": pre,
                                          "recall": rec, "f1_micro":f1_micro,
                                          "f1_macro":f1_macro, "f1_weighted":f1_weighted}, index=[0])
            df_stats = df_stats.append(df_stats_this, ignore_index=True)
    
    return df_stats


## Activity

### Only fridge

In [ ]:
# Video Only
evaluate_model(df_fridge, 'activity')

In [ ]:
# Network Only
evaluate_model(df_fridge, 'activity', ft_type="feat_netml")

#### Fridge (4-class)

In [ ]:
evaluate_model(df_fridge.replace(['put_back_item', 'take_out_item'], 'open_close_fridge'), 'activity')

In [ ]:
evaluate_model(df_fridge.replace(['put_back_item', 'take_out_item'], 'open_close_fridge'), 'activity', ft_type="feat_netml")

### Only washer

In [ ]:
evaluate_model(df_washer, 'activity')

In [ ]:
evaluate_model(df_washer, 'activity', ft_type="feat_netml")

### Only Alexa

In [ ]:
evaluate_model(df_alexa, 'activity')

In [ ]:
evaluate_model(df_alexa, 'activity', ft_type="feat_netml")

### Only nestcam

In [ ]:
evaluate_model(df_nestcam, 'activity')

In [ ]:
evaluate_model(df_nestcam, 'activity', ft_type="feat_netml")

### Only vacuum 

In [ ]:
evaluate_model(df_vacuum, 'activity')

In [ ]:
evaluate_model(df_vacuum, 'activity', ft_type="feat_netml")

### Only vacuum (home)

In [ ]:
evaluate_model(df_vacuum_home, 'activity')

In [ ]:
evaluate_model(df_vacuum_home, 'activity', ft_type="feat_netml")

## User

In [ ]:
evaluate_model(df_labeled.replace('1', '7'), 'user')

In [ ]:
evaluate_model(df_labeled.replace('1', '7'), 'user', ft_type='feat_netml')

## Device

In [ ]:
evaluate_model(df_labeled, 'device')

In [ ]:
evaluate_model(df_labeled, 'device', ft_type='feat_netml')

# Feature Fusion

In [ ]:
def evaluate_model_concact(df_labeled, label, verbose=False, by_user=False):
    classifiers={"RF": RandomForestClassifier(random_state=42),}
    df_labeled["feat"] = list(np.concatenate((np.array(df_labeled["feat_sf"].tolist()), np.array(df_labeled["feat_netml"]).tolist()), axis=1))
    
    if by_user:
        df_labeled = df_labeled.replace('1', '7')
        user_li = df_labeled.user.tolist()
        choice_li = df_labeled.user.unique().tolist()
        for item_ in range(np.shape(choice_li)[0]):
            if choice_li[item_] == '0':
                del choice_li[item_]
        random.seed(42)
        for item_num in range(0, np.shape(user_li)[0]):
            if user_li[item_num] == '0':
                user_li[item_num] = random.choice(choice_li)
        df_labeled['user'] = user_li
        
        if np.shape(df_labeled.user.unique())[0] < 5:
            gkf = GroupKFold(n_splits=np.shape(df_labeled.user.unique())[0])
        else:
            gkf = GroupKFold(n_splits=5)
        index_li = list(gkf.split(df_labeled, groups=df_labeled.user.tolist()))
    else:
        kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
        index_li = list(kf.split(df_labeled))

    if np.shape(df_labeled['device'].unique())[0] > 1:
        device = "combined"
    else:
        device = df_labeled['device'].unique()[0]
    
    df_stats = pd.DataFrame(columns=["classifier", "device", "accuracy", "precision", "recall", "f1_micro", "f1_macro", "f1_weighted", "feature"])

    for key in classifiers:
        clf = classifiers[key]
        Ytrue_li = []
        Ypred_li = []
        for train_index, test_index in index_li:
            df_train = df_labeled.iloc[train_index].sample(frac=0.85, random_state=42)
            df_test = df_labeled.iloc[test_index]
                
            clf.fit(df_train["feat"].tolist(), df_train[label].tolist())
            
            Ytrue = df_test[label].tolist()
            Ypred = clf.predict(df_test["feat"].tolist())

            Ytrue_li.append(Ytrue)
            Ypred_li.append(Ypred)
            if verbose:
                result = permutation_importance(
                    clf, df_test["feat"].tolist(), df_test[label].tolist(), n_repeats=10, random_state=42, n_jobs=2
                )

                feature_sf = ['feature' + str(i) for i in range(400)]
                feature_netml = ["sub_duration_outbound", "pkts_rate_outbound", "bytes_rate_outbound", "np.mean(sizes)_outbound", "np.std(sizes)_outbound", "q1_size_outbound", 
                                   "q2_size_outbound", 'q3_size_outbound', "np.min(sizes)_outbound", 'np.max(sizes)_outbound', "np.mean(iats)_outbound", 'np.std(iats)_outbound', 
                                   "q1_iat_outbound", 'q2_iat_outbound', "q3_iat_outbound", "np.min(iats)_outbound", 'np.max(iats)_outbound', "flow_num_outbound",
                                   "sub_duration_inbound", "pkts_rate_inbound", "bytes_rate_inbound", "np.mean(sizes)_inbound", "np.std(sizes)_inbound", "q1_size_inbound", 
                                   "q2_size_inbound", 'q3_size_inbound', "np.min(sizes)_inbound", 'np.max(sizes)_inbound', "np.mean(iats)_inbound", 'np.std(iats)_inbound', 
                                   "q1_iat_inbound", 'q2_iat_inbound'," q3_iat_inbound", "np.min(iats)_inbound", 'np.max(iats)_inbound', "flow_num_inbound"]
                feature_names = feature_sf + feature_netml

                importances = pd.Series(result.importances_mean, index=feature_names)
                print(key, label)
                print(classification_report(Ypred, df_test[label].tolist()))
            
            acc = accuracy_score(Ytrue, Ypred)
            pre = precision_score(Ytrue, Ypred, average='weighted')
            rec = recall_score(Ytrue, Ypred, average='weighted')
            f1_micro = f1_score(Ytrue, Ypred, average='micro')
            f1_macro = f1_score(Ytrue, Ypred, average='macro')
            f1_weighted = f1_score(Ytrue, Ypred, average='weighted')

            df_stats_this = pd.DataFrame({ "classifier": key, "device": device,
                                          "feature": "feat", 
                                          "accuracy": acc, "precision": pre,
                                          "recall": rec, "f1_micro":f1_micro,
                                          "f1_macro":f1_macro, 
                                          "f1_weighted":f1_weighted}, index=[0])
            df_stats = df_stats.append(df_stats_this, ignore_index=True)
    return df_stats

## Only fridge

In [ ]:
evaluate_model_concact(df_fridge, 'activity')

In [ ]:
evaluate_model_concact(df_fridge.replace(['put_back_item', 'take_out_item'], 'open_close_fridge'), 'activity')

## Only washer

In [ ]:
evaluate_model_concact(df_washer, 'activity')

## Only Alexa

In [ ]:
evaluate_model_concact(df_alexa, 'activity')

## Only nestcam

In [ ]:
evaluate_model_concact(df_nestcam, 'activity')

## Only vacuum

In [ ]:
evaluate_model_concact(df_vacuum, 'activity')

## Only vacuum (home)

In [ ]:
evaluate_model_concact(df_vacuum_home, 'activity')

# Soft voting

In [ ]:
def fit_multiple_estimators(classifiers, X_list, y, sample_weights = None):

    # Convert the labels `y` using LabelEncoder, because the predict method is using index-based pointers
    # which will be converted back to original data later.
    le_ = LabelEncoder()
    le_.fit(y)
    transformed_y = le_.transform(y)

    # Fit all estimators with their respective feature arrays
    estimators_ = [clf.fit(X, y) if sample_weights is None else clf.fit(X, y, sample_weights) for clf, X in zip([clf for _, clf in classifiers], X_list)]

    return estimators_, le_


def predict_from_multiple_estimator(estimators, label_encoder, X_list, weights = None):
    # Predict 'soft' voting with probabilities
    pred1 = np.asarray([clf.predict_proba(X) for clf, X in zip(estimators, X_list)])
    pred2 = np.average(pred1, axis=0, weights=weights)
    pred = np.argmax(pred2, axis=1)

    # Convert integer predictions to original labels:
    return label_encoder.inverse_transform(pred)

def simulate_loss_by_portion_soft(df, portion, loss_type):
    df_changed = df.sample(frac=portion/100, random_state=42)
    df_unchanged = df.drop(df_changed.index)

    if portion != 0:
        if loss_type == "net":
            df_changed["feat_netml"] = [np.zeros(36)]*np.shape(df_changed)[0]
        elif loss_type == "vid":
            df_changed["feat_sf"] = [np.zeros(400)]*np.shape(df_changed)[0]

    return pd.concat([df_changed, df_unchanged], ignore_index=True)


def evaluate_model_soft_voting(df_labeled, label, by_user = False, loss_type = False, portion=0):
    if by_user:
        df_labeled = df_labeled.replace('1', '7')
        user_li = df_labeled.user.tolist()
        choice_li = df_labeled.user.unique().tolist()
        for item_ in range(np.shape(choice_li)[0]):
            if choice_li[item_] == '0':
                del choice_li[item_]
        random.seed(42)
        for item_num in range(0, np.shape(user_li)[0]):
            if user_li[item_num] == '0':
                user_li[item_num] = random.choice(choice_li)
        df_labeled['user'] = user_li
        
        if np.shape(df_labeled.user.unique())[0] < 5:
            gkf = GroupKFold(n_splits=np.shape(df_labeled.user.unique())[0])
        else:
            gkf = GroupKFold(n_splits=5)
        index_li = list(gkf.split(df_labeled, groups=df_labeled.user.tolist()))
    else:
        kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
        index_li = list(kf.split(df_labeled))

    if np.shape(df_labeled['device'].unique())[0] > 1:
        device = "combined"
    else:
        device = df_labeled['device'].unique()[0]
    
    df_stats = pd.DataFrame(columns=["classifier", "device", "accuracy", "precision", "recall", "f1_micro", "f1_macro", "f1_weighted", "feature"])

    Ytrue_li = []
    Ypred_li = []
    for train_index, test_index in index_li:
        classifiers = [("OvRLogReg", OneVsRestClassifier(LogisticRegression(random_state=42))), 
                   ("RF", RandomForestClassifier(random_state=42)),]
        df_train = df_labeled.iloc[train_index].sample(frac=0.85, random_state=42)
        df_test = df_labeled.iloc[test_index]
    
        X_train_list = [df_train.feat_sf.tolist(), df_train.feat_netml.tolist()]
        
        df_test = simulate_loss_by_portion_soft(df_test, portion, loss_type)
        X_test_list = [df_test.feat_sf.tolist(), df_test.feat_netml.tolist()]

        y_train = df_train[label].tolist()
        y_test = df_test[label].tolist()
        Ytrue = df_test[label].tolist()

        fitted_estimators, label_encoder = fit_multiple_estimators(classifiers, X_train_list, y_train)
        Ypred = predict_from_multiple_estimator(fitted_estimators, label_encoder, X_test_list)

        Ytrue_li.append(Ytrue)
        Ypred_li.append(Ypred)
        
        acc = accuracy_score(Ytrue, Ypred)
        pre = precision_score(Ytrue, Ypred, average='weighted')
        rec = recall_score(Ytrue, Ypred, average='weighted')
        f1_micro = f1_score(Ytrue, Ypred, average='micro')
        f1_macro = f1_score(Ytrue, Ypred, average='macro')
        f1_weighted = f1_score(Ytrue, Ypred, average='weighted')

        df_stats_this = pd.DataFrame({ "classifier": "soft_voting", "device": device,
                                      "feature": "feat", 
                                      "accuracy": acc, "precision": pre,
                                      "recall": rec, "f1_micro":f1_micro,
                                      "f1_macro":f1_macro, "f1_weighted":f1_weighted}, index=[0])
        df_stats = df_stats.append(df_stats_this, ignore_index=True)

    return df_stats

## Only fridge

In [ ]:
evaluate_model_soft_voting(df_fridge, 'activity')

In [ ]:
evaluate_model_soft_voting(df_fridge.replace(['put_back_item', 'take_out_item'], 'open_close_fridge'), 'activity')

## Only washer

In [ ]:
evaluate_model_soft_voting(df_washer, 'activity')

## Only alexa

In [ ]:
evaluate_model_soft_voting(df_alexa, 'activity')

## Only nestcam

In [ ]:
evaluate_model_soft_voting(df_nestcam, 'activity')

## Only vacuum

In [ ]:
evaluate_model_soft_voting(df_vacuum, 'activity')

## Only vacuum (home)

In [ ]:
evaluate_model_soft_voting(df_vacuum_home, 'activity')

# Stacking

In [ ]:
def evaluate_model_stacking(df_labeled, label, verbose=False, number=400, by_user = False):
    df_labeled["feat"] = list(np.concatenate((np.array(df_labeled["feat_sf"].tolist()), np.array(df_labeled["feat_netml"]).tolist()), axis=1))
    
    if by_user:
        df_labeled = df_labeled.replace('1', '7')
        user_li = df_labeled.user.tolist()
        choice_li = df_labeled.user.unique().tolist()
        for item_ in range(np.shape(choice_li)[0]):
            if choice_li[item_] == '0':
                del choice_li[item_]
        random.seed(42)
        for item_num in range(0, np.shape(user_li)[0]):
            if user_li[item_num] == '0':
                user_li[item_num] = random.choice(choice_li)
        df_labeled['user'] = user_li
        
        if np.shape(df_labeled.user.unique())[0] < 5:
            gkf = GroupKFold(n_splits=np.shape(df_labeled.user.unique())[0])
        else:
            gkf = GroupKFold(n_splits=5)
        index_li = list(gkf.split(df_labeled, groups=df_labeled.user.tolist()))
    else:
        kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
        index_li = list(kf.split(df_labeled))
    
    if np.shape(df_labeled['device'].unique())[0] > 1:
        device = "combined"
    else:
        device = df_labeled['device'].unique()[0]
    
    df_stats = pd.DataFrame(columns=["classifier", "device", "accuracy", "precision", "recall", "f1_micro", "f1_macro", "f1_weighted", "feature"])

    Ytrue_li = []
    Ypred_li = []
    for train_index, test_index in index_li:
        clf_1 = OneVsRestClassifier(LogisticRegression(random_state=42))
        clf_2 = RandomForestClassifier(random_state=42)
        fclf = RandomForestClassifier(random_state=42)
        
        df_train = df_labeled.iloc[train_index]
        df_test = df_labeled.iloc[test_index]
        
        df_base_train = df_train.sample(frac=0.85, random_state=42)
        df_meta_train_all = df_train.loc[~df_train.index.isin(df_base_train.index.tolist())]
        
        df_meta_train = df_train.loc[~df_train.index.isin(df_base_train.index.tolist())]
        
        if number == 400:
            df_meta_train = df_meta_train.groupby(label).sample(frac=1, random_state=42)
        else:
            df_meta_train = df_meta_train_all.sample(n=number, random_state=42, replace=True)
    
        X_base_train = np.array(df_base_train.feat.tolist())

        
        X_meta_train = np.array(df_meta_train.feat.tolist())
        X_test = np.array(df_test.feat.tolist())

        y_base_train = np.array(df_base_train[label].tolist())
        y_meta_train = np.array(df_meta_train[label].tolist())
        y_test = np.array(df_test[label].tolist())
        
        clf_1.fit(X_base_train[:, :400], y_base_train)
        clf_2.fit(X_base_train[:, 400:], y_base_train)
        
        ft1_meta_train = clf_1.predict_proba(X_meta_train[:, :400])
        ft2_meta_train = clf_2.predict_proba(X_meta_train[:, 400:])
        
        ft_meta_train = np.concatenate((ft1_meta_train, ft2_meta_train), axis=1)

        fclf.fit(ft_meta_train, y_meta_train)
        
        Ytrue = np.array(df_test[label].tolist())
        ft_test = np.concatenate((clf_1.predict_proba(X_test[:, :400]), clf_2.predict_proba(X_test[:, 400:])), axis=1)
        Ypred = fclf.predict(ft_test)

        Ytrue_li.append(Ytrue)
        Ypred_li.append(Ypred)
        
        if verbose:
            result = permutation_importance(
                    fclf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
                )

            feature_sf = ['feature' + str(i) for i in range(400)]
            feature_netml = ["sub_duration_outbound", "pkts_rate_outbound", "bytes_rate_outbound", "np.mean(sizes)_outbound", "np.std(sizes)_outbound", "q1_size_outbound", 
                               "q2_size_outbound", 'q3_size_outbound', "np.min(sizes)_outbound", 'np.max(sizes)_outbound', "np.mean(iats)_outbound", 'np.std(iats)_outbound', 
                               "q1_iat_outbound", 'q2_iat_outbound', "q3_iat_outbound", "np.min(iats)_outbound", 'np.max(iats)_outbound', "flow_num_outbound",
                               "sub_duration_inbound", "pkts_rate_inbound", "bytes_rate_inbound", "np.mean(sizes)_inbound", "np.std(sizes)_inbound", "q1_size_inbound", 
                               "q2_size_inbound", 'q3_size_inbound', "np.min(sizes)_inbound", 'np.max(sizes)_inbound', "np.mean(iats)_inbound", 'np.std(iats)_inbound', 
                               "q1_iat_inbound", 'q2_iat_inbound'," q3_iat_inbound", "np.min(iats)_inbound", 'np.max(iats)_inbound', "flow_num_inbound"]
            feature_names = feature_sf + feature_netml

            importances = pd.Series(result.importances_mean, index=feature_names)
        
        acc = accuracy_score(Ytrue, Ypred)
        pre = precision_score(Ytrue, Ypred, average='weighted')
        rec = recall_score(Ytrue, Ypred, average='weighted')
        f1_micro = f1_score(Ytrue, Ypred, average='micro')
        f1_macro = f1_score(Ytrue, Ypred, average='macro')
        f1_weighted = f1_score(Ytrue, Ypred, average='weighted')

        df_stats_this = pd.DataFrame({ "classifier": "stacking", "device": device,
                                      "feature": "feat", 
                                      "accuracy": acc, "precision": pre,
                                      "recall": rec, "f1_micro":f1_micro,
                                      "f1_macro":f1_macro, 
                                      "f1_weighted":f1_weighted}, index=[0])
        df_stats = df_stats.append(df_stats_this, ignore_index=True)
    return df_stats

## Only fridge

In [ ]:
evaluate_model_stacking(df_fridge, 'activity')

In [ ]:
evaluate_model_stacking(df_fridge.replace(['put_back_item', 'take_out_item'], 'open_close_fridge'), 'activity')

## Only washer

In [ ]:
evaluate_model_stacking(df_washer, 'activity')

## Only alexa

In [ ]:
evaluate_model_stacking(df_alexa, 'activity')

## Only nestcam

In [ ]:
evaluate_model_stacking(df_nestcam, 'activity')

## Only vacuum 

In [ ]:
evaluate_model_stacking(df_vacuum, 'activity')

## Only vacuum (home)

In [ ]:
evaluate_model_stacking(df_vacuum_home, 'activity')